# ESE 415 Final Project - Analyzing Data Scientist Salaries

## By: Abigail Alpert and Kevin Yan

### Dataset Columns
1. **work_year** : The year the salary was paid.

2. **experience_level** : The experience level in the job during the year

3. **employment_type** : The type of employment for the role

4. **job_title** : The role worked in during the year.

5. **salary** : The total gross salary amount paid.

6. **salary_currency** : The currency of the salary paid as an ISO 4217 currency code.

7. **salaryinusd** : The salary in USD

8. **employee_residence** : Employee's primary country of residence in during the work year as an ISO 3166 country code.

9. **remote_ratio** : The overall amount of work done remotely

10. **company_location** : The country of the employer's main office or contracting branch

11. **company_size** : The median number of people that worked for the company during the year

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv('ds_salaries.csv', index_col = 0)
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...
602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


First off, basic data cleaning (dropping duplicates and NA's).

In [3]:
df = df.drop_duplicates()
df = df.dropna(subset=['salary_in_usd']) 

Now let's create a couple flags/binary variables that could be useful for predictions

In [4]:
df['is_manager'] = df['job_title'].str.contains('Manager|Lead|Director|Head', case=False).astype(int)
df['is_remote'] = (df['remote_ratio'] == 100).astype(int)
df['is_hybrid'] = ((df['remote_ratio'] > 0) & (df['remote_ratio'] < 100)).astype(int)
df['same_country'] = (df['employee_residence'] == df['company_location']).astype(int)

In [5]:
print("Unique Job Titles:",len(df['job_title'].unique()))
print("Unique Employee Residences:",len(df['employee_residence'].unique()))
print("Unique Company Locations:",len(df['company_location'].unique()))

Unique Job Titles: 50
Unique Employee Residences: 57
Unique Company Locations: 50


Unfortunately, it seems like these categorical variables have many possibilities, so performing one-hot encoding on these columns may create too many columns to be feasible. We will have to find other ways to deal with these columns, as we believe that the job title will play a strong role in predicting salary.

With that being said, let's handle the lower-cardinal categorical features with one-hot encoding. Note that we ignore salary_currency as we do not think it will be predictive given we have the salary_in_usd column.

In [8]:
low_card_cat_features = [
    'experience_level',  # EN, MI, SE, EX
    'employment_type',   # FT, PT, CT, FL
    'company_size',     # S, M, L
]
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), low_card_cat_features)
    ],
    remainder='passthrough'
)

X_processed = preprocessor.fit_transform(df)


cat_encoder = preprocessor.named_transformers_['cat']
feature_names = cat_encoder.get_feature_names_out(low_card_cat_features)

all_feature_names = list(feature_names) + [
    col for col in df.columns 
    if col not in low_card_cat_features
]

processed_df = pd.DataFrame(X_processed, columns=all_feature_names)
processed_df

,experience_level_EN,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_CT,employment_type_FL,employment_type_FT,employment_type_PT,company_size_L,company_size_M,...,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,is_manager,is_remote,is_hybrid,same_country
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,70000,EUR,79833,DE,0,DE,0,0,0,1
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,260000,USD,260000,JP,0,JP,0,0,0,1
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,85000,GBP,109024,GB,50,GB,0,0,1,1
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,20000,USD,20000,HN,0,HN,0,0,0,1
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,150000,USD,150000,US,50,US,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,154000,USD,154000,US,100,US,0,1,0,1
561,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,126000,USD,126000,US,100,US,0,1,0,1
562,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,129000,USD,129000,US,0,US,0,0,0,1
563,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,150000,USD,150000,US,100,US,0,1,0,1
